In [18]:
from Sample import Sample
from _01_preprocess import preprocess
from _02_register import FeatureExtractor
import matplotlib.pyplot as plt
from xtiff import to_tiff
import os
import glob
import pandas as pd
import numpy as np
import shutil
import tifffile as tiff
from skimage.transform import resize

In [ ]:
dataset_path = "/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/TU"
base_path="/data_isilon_main/isilon_images/10_MetaSystems/MetaSystemsData/Multimodal_Imaging_Daria/_Data_Analysis/_tmp_daria/Image_analysis/20221010_Steinbock_Mesmer_IFbased/PT"

samples = pd.read_csv(base_path + "/RoI_to_exclude.csv", sep=';')

In [21]:
for index, row in samples.iterrows():

    sample_path = glob.glob(dataset_path + '/*{}*'.format(row['Sample']))[0]
    S = Sample(sample_path)

    if not os.path.exists(os.path.join(sample_path, 'IF_IMC_lowres')):
        os.makedirs(os.path.join(sample_path, 'IF_IMC_lowres'))

    for i, roi in enumerate(S.rois):
        if not row['exclude'] == "no" or row['exclude'] == "maybe":
            if str(roi.roi_num) in ((row['exclude'].split(','))):
                continue

        print('RoI_' + str(roi.roi_num) + ': ')
        ppIF = preprocess(roi.if_nuc)
        ppIMC = preprocess(roi.imc_nuc)

        ex = FeatureExtractor("sift")
        ex(ppIMC, ppIF)
        ex.match()
        ex.estimate()

        #Apply transformation function to all IF and IMC images
        imc_cut, if_cut = ex.warp(im0=roi.imc_imgs, im1=roi.if_imgs)
        mc_image = np.concatenate((imc_cut, if_cut), axis=0) 

        mc_image = np.expand_dims(mc_image, axis=(0, 1, 5))

        if roi.roi_num <10:
            roi_prefix = "_ROI_00"
        else:
            roi_prefix = "_ROI_0"

        title= S.sample_name + roi_prefix + str(roi.roi_num) + "_IF_IMC.tiff"
        to_tiff(
            mc_image,
            os.path.join(base_path, 'img', title),
            image_name=title,
            channel_names=[*roi.imc_marker, *roi.if_marker],
            pixel_size=1.0,
            pixel_depth=1.0,
        )

        to_tiff(
            mc_image,
            os.path.join(sample_path, 'IF_IMC_lowres', title),
            image_name=title,
            channel_names=[*roi.imc_marker, *roi.if_marker],
            pixel_size=1.0,
            pixel_depth=1.0,
        )

IndexError: list index out of range